In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
import random as rn
np.random.seed(37)
rn.seed(1254)
tf.random.set_seed(89)
import os
os.environ['PYTHONHASHSEED'] = '0'
from datetime import datetime
from pathlib import Path
import matplotlib.pyplot as plt
import tensorflow as tf
from watch_recognition.models import build_backbone
from watch_recognition.reports import log_distances, run_on_image_debug, generate_report_for_keypoints


plt.style.use("dark_background")
%matplotlib inline


EPOCHS = 300
image_size = (224, 224)

In [ ]:
base_model = tf.keras.applications.EfficientNetB0(
        weights="imagenet",  # Load weights pre-trained on ImageNet.
        input_shape=(*image_size, 3),
        include_top=False,
    )
for layer in base_model.layers:
    if 'project_conv' in layer.name:
        print(layer.name, layer.output.shape)

In [ ]:
base_model = build_backbone(image_size)
# base_model.trainable = False

In [ ]:
# build model

inputs = tf.keras.Input(shape=(*image_size, 3), )
x = base_model(inputs)
# x = tf.keras.layers.Conv2D(
#     filters=128, kernel_size=3, strides=1, padding="same", activation='relu'
# )(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPooling2D(2, strides=1,padding='same')(x)
x = tf.keras.layers.Conv2D(
    filters=64, kernel_size=3, strides=1, padding="same", activation='relu'
)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPooling2D(2, strides=1,padding='same')(x)
output = tf.keras.layers.Conv2D(
    filters=4, kernel_size=1, strides=1, padding="same", activation='sigmoid'
)(x)


model = tf.keras.models.Model(
    inputs=inputs, outputs=output
)

In [ ]:
model.summary()


In [ ]:
model.output.shape[1:]

In [ ]:
from watch_recognition.data_preprocessing import load_keypoints_data_2

X, y = load_keypoints_data_2(
    Path("../download_data/keypoints/train"),
    mask_size=(14,14),
    extent=(3,3)
)
X.shape, y.shape

In [ ]:
from watch_recognition.augmentations import aug_fn, process_data, set_shapes
from functools import partial


dataset = tf.data.Dataset.from_tensor_slices((X, y))
AUTOTUNE = tf.data.experimental.AUTOTUNE


ds_alb = dataset.map(partial(process_data, mask_size=(14, 14), image_size=(224, 224)),
                     num_parallel_calls=AUTOTUNE)

ds_alb = ds_alb.map(
    partial(set_shapes, img_shape=(224, 224, 3), masks_shape=(14,14, 4)),
    num_parallel_calls=AUTOTUNE).batch(32).prefetch(AUTOTUNE)
ds_alb

In [ ]:
X_val, y_val = load_keypoints_data_2(
    Path("../download_data/keypoints/validation"),
    mask_size=(14,14),
    extent=(3,3)
)
X_val.shape, y_val.shape

In [ ]:

from watch_recognition.models import custom_focal_loss

optimizer = tf.keras.optimizers.Adam()
# loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=1e-8)
loss = custom_focal_loss
model.compile(loss=loss, optimizer=optimizer)


start = datetime.now()
TYPE='keypoint'
MODEL_NAME='efficientnetb0'
logdir = f"tensorboard_logs/{TYPE}/{MODEL_NAME}/run_{start.timestamp()}"
print(logdir)
file_writer_distance_metrics_train = tf.summary.create_file_writer(logdir + "/train")
file_writer_distance_metrics_validation = tf.summary.create_file_writer(logdir + "/validation")

model_path = f'models/{TYPE}/{MODEL_NAME}/run_{start.timestamp()}.h5'
model.fit(
    ds_alb,
    epochs=EPOCHS,
    validation_data=(X_val, y_val),
    callbacks=[
        tf.keras.callbacks.TensorBoard(
            log_dir=logdir,
            update_freq="epoch",
        ),
        tf.keras.callbacks.ModelCheckpoint(
                filepath=model_path,
                save_weights_only=False,
                monitor='val_loss',
                save_best_only=True,
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss', factor=0.8,
                              patience=20, min_lr=1e-6),
        # tf.keras.callbacks.EarlyStopping(
        #     monitor="val_loss",
        #     restore_best_weights=True,
        #     patience=15,
        # ),
        tf.keras.callbacks.LambdaCallback(
            on_epoch_end=partial(log_distances, X=X, y=y,
                                 file_writer=file_writer_distance_metrics_train, model=model)),
        tf.keras.callbacks.LambdaCallback(
            on_epoch_end=partial(log_distances, X=X_val, y=y_val,
                                 file_writer=file_writer_distance_metrics_validation, model=model)),
    ]
)
elapsed = (datetime.now() - start).seconds
print(f"total training time: {elapsed / 60} minutes, average: {elapsed/60/EPOCHS} minutes/epoch")


In [ ]:
run_on_image_debug(model, X[0], y[0], show_grid=False)


In [ ]:
path = Path("../example_data/Zrzut ekranu 2021-08-25 o 22.24.14.png")
test_image = tf.keras.preprocessing.image.load_img(
    path, "rgb", target_size=image_size, interpolation="bicubic",
)
test_image_np = tf.keras.preprocessing.image.img_to_array(test_image)

In [ ]:
run_on_image_debug(model, test_image_np, show_grid=False)

In [ ]:
generate_report_for_keypoints(model, X, y, show_top_n_errors=5)

In [ ]:
generate_report_for_keypoints(model, X_val, y_val, show_top_n_errors=5)

In [ ]:
model_path = "./models/keypoint/efficientnetb0/run_1629928881.918389.h5"
loaded_model = tf.keras.models.load_model(model_path, compile=False)

In [ ]:
for path in [
    Path("../example_data/Zrzut ekranu 2021-08-25 o 22.24.14.png"),
    Path("../example_data/Zrzut ekranu 2021-08-25 o 22.24.24.png"),
    Path("../example_data/test-image-2.jpg"),
]:
    test_image = tf.keras.preprocessing.image.load_img(
        path, "rgb", target_size=image_size, interpolation="bicubic",
    )
    test_image_np = tf.keras.preprocessing.image.img_to_array(test_image)

    run_on_image_debug(loaded_model, test_image_np, show_grid=False)

In [ ]:
generate_report_for_keypoints(loaded_model, X, y, show_top_n_errors=5)

In [ ]:
generate_report_for_keypoints(loaded_model, X_val, y_val, show_top_n_errors=5)